In [14]:
import pandas as pd

In [15]:
df1 = pd.read_excel(r"~/Documents/NeenOpal/Task1.xlsx",sheet_name="data1")

df2 = pd.read_excel(r"~/Documents/NeenOpal/Task1.xlsx",sheet_name="data2")

In [16]:
print(df1)

            Order ID       Product ID  Qty
0     CA-2014-100090  FUR-TA-10003715    3
1     CA-2014-100090  OFF-BI-10001597    6
2     CA-2014-100293  OFF-PA-10000176    6
3     CA-2014-100328  OFF-BI-10000343    1
4     CA-2014-100363  OFF-FA-10000611    2
...              ...              ...  ...
9472  US-2017-169551  OFF-PA-10004100    3
9473  US-2017-169551  OFF-ST-10004835    3
9474  US-2017-169551  TEC-AC-10002018    3
9475  US-2017-169551  TEC-AC-10003033    2
9476  US-2017-169551  TEC-PH-10001363    2

[9477 rows x 3 columns]


In [17]:
import psycopg2

In [18]:
conn = psycopg2.connect(
    dbname='neenopal',  
    user='postgres',   
    password='postgres',
    host='localhost',        
    port='5432'              
)

In [19]:
print(conn)

<connection object at 0x118d33ca0; dsn: 'user=postgres password=xxx dbname=neenopal host=localhost port=5432', closed: 0>


In [20]:


cursor = conn.cursor()

In [21]:
cursor.execute("""
    CREATE TABLE table1 (
    orderID VARCHAR(255),
    productID VARCHAR(255),
    quantity INTEGER
);
""");


In [22]:
conn.commit()

In [23]:
print(df1.iloc[0])

Order ID       CA-2014-100090
Product ID    FUR-TA-10003715
Qty                         3
Name: 0, dtype: object


In [26]:

for index, row in df1.iterrows():
#     print(row)
    cursor.execute(
        "INSERT INTO table1 (orderID, productID,quantity) VALUES (%s, %s, %s)",
        (row['Order ID'], row['Product ID'], row['Qty'])
    )

conn.commit()

In [27]:
cursor.execute("""
    CREATE TABLE table2 (
    orderID VARCHAR(255),
    productID VARCHAR(255),
    quantity INTEGER
);
""");

conn.commit()

In [28]:
for index, row in df2.iterrows():
#     print(row)
    cursor.execute(
        "INSERT INTO table2 (orderID, productID,quantity) VALUES (%s, %s, %s)",
        (row['Order ID'], row['Product ID'], row['Qty'])
    )

conn.commit()

In [29]:

# in_df1_not_in_df2 = 0

# for i in range(len(df1)):
#     cursor.execute(
#         """
#         select * from table2 where orderID = %s and productID = %s
#         """,
#         (df1.iloc[i]['Order ID'],df1.iloc[i]['Product ID'])
#     )
#     t = cursor.fetchall()
#     if len(t) == 0:
#         in_df1_not_in_df2 = in_df1_not_in_df2 + 1
        

# print(in_df1_not_in_df2)

506


In [42]:

cursor.execute("""
select * from (
select table1.orderid  as orderid, table2.productid as productid 
FROM table1 left JOIN table2 on table1.orderid  = table2.orderid and table1.productid = table2.productid 
) as temptable

where productid is null
""")

rows = cursor.fetchall()

print('Entries that are in table 1 but not in table 2: ',end='')
print(len(rows))


Entries that are in table 1 but not in table 2: 506


In [43]:
cursor.execute("""
select * from (
select table2.orderid  as orderid, table1.productid as productid, table2.quantity as quantity 
FROM table2 left JOIN table1 on table2.orderid  = table1.orderid and table2.productid = table1.productid 
) as temptable

where productid is null
""")

rows2 = cursor.fetchall()

print('Entries that are in table 2 but not in table 1: ',end='')
print(len(rows2))


Entries that are in table 2 but not in table 1: 508


In [44]:
sum = 0

# print(rows2[0])


for i in range(len(rows2)):
    sum = sum + rows2[i][2]
    
    
print('sum of quantities of all entries from table 2 but not in table 1: ',end = '')
print(sum)

sum of quantities of all entries from table 2 but not in table 1: 1956


In [45]:

# total number of unique records 

print('total number of unique records: ',end='')
print(len(rows) + len(rows2) + len(df1) - len(rows))

total number of unique records: 9985


In [46]:
cursor.close()

In [47]:
conn.close()